In [ ]:
import sys
from pathlib import Path

AVES_ROOT = Path("../..") if not "google.colab" in sys.modules else Path("aves_git")

EOD_PATH = AVES_ROOT / "data" / "external" / "EOD_STGO"
EOD_PATH

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import geopandas as gpd

# esto configura la calidad de la imagen. dependerá de tu resolución. el valor por omisión es 80
mpl.rcParams["figure.dpi"] = 120
# esto depende de las fuentes que tengas instaladas en el sistema.
#mpl.rcParams["font.family"] = "Fira Sans Extra Condensed"


In [ ]:
zones = gpd.read_file(AVES_ROOT / "data" / "processed" / "scl_zonas_urbanas.json")
zones.head()

In [ ]:
from aves.data import census

comunas = census.read_census_map('comuna', path=AVES_ROOT / "data" / "external" / "censo_2017_R13")

In [ ]:
comunas_urbanas = comunas[comunas['COMUNA'].isin(zones['Com'].unique())].drop('NOM_COMUNA', axis=1).copy()
comunas_urbanas['NombreComuna'] = comunas_urbanas['COMUNA'].map(dict(zip(zones['Com'], zones['Comuna'])))
comunas_urbanas.plot(facecolor="none", edgecolor="#abacab")

In [ ]:
from aves.features.geo import clip_area_geodataframe
comunas_urbanas = clip_area_geodataframe(comunas_urbanas, zones.total_bounds, buffer=0.05)
comunas_urbanas.plot()

In [ ]:
from aves.data import eod

viajes = (
    eod.read_trips(EOD_PATH)
    .merge(eod.read_people(EOD_PATH))
    .merge(eod.read_homes(EOD_PATH))
)

viajes["PesoLaboral"] = viajes["FactorLaboralNormal"] * viajes["Factor_LaboralNormal"]

viajes = viajes[pd.notnull(viajes["PesoLaboral"])]

viajes.columns

In [ ]:
# matriz origen-destino

In [ ]:
matriz = (
    viajes[
        (viajes["Proposito"] == "Al trabajo")
        & (viajes["ComunaOrigen"].isin(comunas_urbanas["NombreComuna"]))
        & (viajes["ComunaDestino"].isin(comunas_urbanas["NombreComuna"]))
    ]
    .groupby(["ComunaOrigen", "ComunaDestino"])
    .agg(n_viajes=("PesoLaboral", "sum"))
    .reset_index()
)

matriz.head()

In [ ]:
from aves.features.utils import normalize_rows

fig, ax = plt.subplots(figsize=(12, 9))

sns.heatmap(
    matriz.set_index(["ComunaOrigen", "ComunaDestino"])["n_viajes"]
    .unstack(fill_value=0)
    .pipe(normalize_rows),
    cmap="inferno_r",
    linewidth=1,
)

In [ ]:
from aves.models.network import Network

od_network = Network.from_edgelist(
    # graficamos los viajes más representativos
    matriz[matriz["n_viajes"] > matriz["n_viajes"].quantile(0.75)],
    source="ComunaOrigen",
    target="ComunaDestino",
    weight="n_viajes",
)

In [ ]:
from aves.visualization.networks import NodeLink

nodelink = NodeLink(od_network)
nodelink.layout_nodes(method='geographical', geodataframe=comunas_urbanas, node_column='NombreComuna')

In [ ]:
nodelink.set_edge_drawing('origin-destination')

In [ ]:
nodelink.set_node_drawing("plain", weights='in_degree')

In [ ]:
from aves.visualization.figures import figure_from_geodataframe

fig, ax = figure_from_geodataframe(zones, height=7)

# contexto
zones.plot(ax=ax, facecolor='#efefef', edgecolor='white', zorder=0)
comunas_urbanas.plot(ax=ax, facecolor='none', edgecolor='#abacab', zorder=1)

nodelink.plot(ax, nodes=dict(color='white', edgecolor='black', node_size=150, alpha=0.95), edges=dict(alpha=0.5), zorder=2)

ax.set_title('Viajes al trabajo en Santiago (en días laborales, EOD 2012)')

fig.tight_layout()

In [ ]:
matriz_zonas = (
    viajes[
        (viajes["Proposito"].isin(["Al trabajo", "Al estudio"]))
        & (viajes["ZonaOrigen"] != viajes["ZonaDestino"])
        & (viajes["ZonaOrigen"].isin(zones['ID']))
        & (viajes["ZonaDestino"].isin(zones['ID']))
    ]
    .groupby(["ComunaOrigen", "ComunaDestino", "ZonaOrigen", "ZonaDestino"])
    .agg(n_viajes=("PesoLaboral", "sum"))
    .sort_values("n_viajes", ascending=False)
    .assign(cumsum_viajes=lambda x: x["n_viajes"].cumsum())
    .assign(cumsum_viajes=lambda x: x["cumsum_viajes"] / x["cumsum_viajes"].max())
    .reset_index()
)

matriz_zonas.head()


In [ ]:
zone_od_network = Network.from_edgelist(
    matriz_zonas[matriz_zonas["cumsum_viajes"] <= 0.5],
    source="ZonaOrigen",
    target="ZonaDestino",
    weight="n_viajes",
)  # .largest_connected_component(directed=True)
zone_od_network.network, zone_od_network.num_vertices, zone_od_network.num_edges


In [ ]:
merged_zones = zones.reset_index().dissolve("ID")
merged_zones.head()


In [ ]:
zone_nodelink = NodeLink(zone_od_network)
zone_nodelink.layout_nodes(method="geographical", geodataframe=merged_zones)
zone_nodelink.set_node_drawing("plain", weights="in_degree")
zone_nodelink.set_edge_drawing(method="origin-destination")


In [ ]:
fig, ax = figure_from_geodataframe(zones, height=7)

# contexto
zones.plot(ax=ax, facecolor="#efefef", edgecolor="white", zorder=0)
comunas_urbanas.plot(ax=ax, facecolor="none", edgecolor="#abacab", zorder=1)

zone_nodelink.plot(
    ax,
    nodes=dict(color="white", edgecolor="black", node_size=150, alpha=0.95),
    edges=dict(alpha=0.5),
    zorder=2,
)

ax.set_title("Viajes al trabajo en Santiago (en días laborales, EOD 2012)")

fig.tight_layout()


In [ ]:
import pyrosm

# redes con OSM
OSM_PATH = AVES_ROOT / "data" / "external" / "OSM"
osm = pyrosm.OSM(str(OSM_PATH / 'clipped-scl-osm.pbf'))

In [ ]:
network_nodes, network_edges  = osm.get_network(nodes=True)

In [ ]:
network_nodes

In [ ]:
network_edges

In [ ]:
network_edges[['u', 'v', 'length']]

In [ ]:
walk_network = Network.from_edgelist(
    network_edges,
    source="u",
    target="v",
    weight="length",
)

In [ ]:
walk_nodelink = NodeLink(walk_network)
walk_nodelink.layout_nodes(
    method="geographical", geodataframe=network_nodes, node_column="id"
)


In [ ]:
fig, ax = figure_from_geodataframe(zones, height=7)

comunas_urbanas.plot(ax=ax, facecolor="none", edgecolor="#abacab", zorder=1)

walk_nodelink.plot(
    ax,
    nodes=dict(color="black", edgecolor="none", node_size=1, alpha=1.0),
    edges=dict(alpha=0.25, linewidth=0.25),
    zorder=2,
)

fig.tight_layout()


In [ ]:
fig, ax = figure_from_geodataframe(zones[zones["NOM_COMUNA"] == "SANTIAGO"], height=7)

comunas_urbanas.plot(ax=ax, facecolor="none", edgecolor="#abacab", zorder=1)

walk_nodelink.plot(
    ax,
    nodes=dict(color="black", edgecolor="none", node_size=1, alpha=1.0),
    edges=dict(alpha=0.5, linewidth=0.25),
    zorder=2,
)

fig.tight_layout()


In [ ]:
# red accesible desde un punto caminando durante 15 minutos??
query_point = (-70.6651616, -33.4582698)


In [ ]:
import shapely.geometry

query_point_node_id = network_nodes.distance(
    shapely.geometry.Point(query_point)
).idxmin()
query_point_node_id


In [ ]:
graph_node_id = walk_network.node_map[network_nodes.loc[query_point_node_id]["id"]]
graph_node_id


In [ ]:
import graph_tool.search


class Visitor(graph_tool.search.BFSVisitor):
    def __init__(self, node_id, edge_weight, pred, dist):
        self.pred = pred
        self.dist = dist
        self.cost = edge_weight

        self.root = node_id
        self.dist[node_id] = 0

        self.next_ring = dict()
        self.visited = set()
        self.visited.add(node_id)

    def discover_vertex(self, u):
        self.next_ring[u] = self.dist[u]

    def examine_vertex(self, u):
        pass

    def tree_edge(self, e):
        self.pred[e.target()] = int(e.source())

        cost = self.dist[e.source()] + self.cost[e]

        # TODO: quizás hay que seleccionar un costo porque hay varias maneras de llegar
        if not e.target() in self.visited:
            self.dist[e.target()] = cost
            self.visited.add(e.target())

    def finish_vertex(self, u):
        del self.next_ring[u]

        if all(cost > 1500 for cost in self.next_ring.values()):
            for node in self.next_ring.keys():
                self.dist[node] = -1
            raise graph_tool.search.StopSearch()


dist = walk_network.graph.new_vertex_property("int", val=2000)
pred = walk_network.graph.new_vertex_property("int64_t")

graph_tool.search.bfs_search(
    walk_network.graph,
    graph_node_id,
    Visitor(graph_node_id, walk_network._edge_weight, pred, dist),
)


In [ ]:
access_network = walk_network.subgraph(vertex_filter=dist.ma < 2000)
access_nodelink = NodeLink(access_network)
access_nodelink.layout_nodes(
    method="geographical", geodataframe=network_nodes, node_column="id"
)


In [ ]:
fig, ax = figure_from_geodataframe(comunas[comunas["NOM_COMUNA"] == "SANTIAGO"])
zones.plot(ax=ax, color="none", edgecolor="black", linewidth=0.5)
ax.plot([query_point[0]], [query_point[1]], marker="x", color="red", zorder=10)
access_nodelink.plot_edges(ax=ax)
